# CNN ile Fruits 360 Veri Setini Siniflandiralim

### Gerekli kutuphanelerimiz

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.image_utils import load_img, img_to_array
import matplotlib.pyplot as plt
from glob import glob

%matplotlib inline

### Egitim Veri Setimizin Bilgisayardaki Yolu

In [ ]:
train_path = 'C:/Users/mehmet/Desktop/Fruits 360/fruits-360_dataset/fruits-360/Training/'
test_path = 'C:/Users/mehmet/Desktop/Fruits 360/fruits-360_dataset/fruits-360/Test/'

### Ornek Resim yukleyelim

In [ ]:
img = load_img(train_path + 'Apple Braeburn/0_100.jpg')

In [ ]:
plt.imshow(img)
plt.axis('off')

In [ ]:
img_test = load_img(test_path + 'Apple Braeburn/69_100.jpg')

In [ ]:
plt.imshow(img_test)
plt.axis('off')

### Resmimizin Boyutlari

In [ ]:
x = img_to_array(img)
x.shape # Width, Height, 3D Resim old. belli eden derinlik

### Kac tane output old. ogrenelim

In [ ]:
class_name = glob(train_path + '/*')
number_of_class = len(class_name)
number_of_class

## Modelimizi Olusturalim

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 3))) # Filtre Sayisi, Filtre Boyutu, Resmin Boyutlari
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # Havuzlama Katmani Boyutlari

model.add(Conv2D(64, (3, 3)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(units=1024))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))
#model.add(BatchNormalization())

model.add(Dense(units=number_of_class))
model.add(Activation('softmax'))

### Modelimizi Derleyelim

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
batch_size = 32 # Tek bir seferde 32 resim ile egitilsin

## Train ve Test veri setlerimizi belirleyelim

#### Her bir kategoride 400 üstü resim old. gore bu resmi cogaltmak icin Data Augmentation/ImageDataGenerator/Resimleri Coklayalim.

### Image Data Generator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, # scaling(0-1) arasina cek
                                  shear_range=0.3, # resmi belirttigimiz deger kadar dondur
                                  horizontal_flip=True, # yatay bir sekilde dondur
                                  zoom_range=0.3, # belirlenen deger kadar yakinlastirir
                                  )

test_datagen = ImageDataGenerator(rescale=1./255) # test edecegimiz resimleri scale etsek yeter aksi halde resim bozulursa pek dogru sonuclar elde edemeyiz

train_generator = train_datagen.flow_from_directory(directory=train_path, # Siniflandirilmis Resimlerimizin bulundugu Ana Dizinimiz
                                                    target_size=(100, 100), # Resimlerimizin Boyutlari
                                                    batch_size=batch_size, # Bir seferde alinacak resim miktari
                                                    color_mode='rgb', # Resimlerimizin formati
                                                    class_mode='categorical', # birden fazla sinifimizin old. belirttik
                                                   )

test_generator = test_datagen.flow_from_directory(directory=test_path,
                                                 target_size=(100, 100),
                                                 batch_size=batch_size,
                                                 color_mode='rgb',
                                                 class_mode='categorical')

## Modelimizi Egitelim

In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch= 1600 // batch_size, # bir epoch boyunca 50 defa verimizi train edelim
                              # bizim baslangicta 400 resmimiz var bunu batch_size'e bolunce 1 epoch'da 12 defa train eder.
                              # Peki bu diger resimler nerden geldi?
                              # 50*32=1600 1200 civari resim ImageDataGenerator'den geliyor.
                              epochs=100,
                              validation_data=test_generator, # validation_generator
                              validation_steps=800
                             )

### Modelimizi Degerlendirelim - Model Evaluation

#### Modeli Kaydedelim

In [ ]:
model.save_weights('deneme_h5')

### Loss Function

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['loss'], label='Train Loss Value')
plt.plot(history.history['val_loss'], label='Train Validation Loss Value')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy Value')
plt.plot(history.history['val_accuracy'], label='Train Validation Accuracy Value')
plt.legend()
plt.show()

### History degerlerini json formatinda kaydedelim 

In [ ]:
import json

In [ ]:
with open('cnn_fruits-360_hist.json', 'w') as f:
    json.dump(history.history, f)

#### History Degerlerini Yukleyelim

In [ ]:
import codecs

In [ ]:
with codecs.open('cnn_fruits-360_hist.json', 'r', encoding='utf-8') as f:
    h = json.loads(f.read())

In [ ]:
h

In [ ]:
plt.plot(h['loss'], label='Train Loss Value')
plt.plot(h['val_loss'], label='Train Validation Loss Value')
plt.legend()
plt.show()

In [ ]:
plt.plot(h['accuracy'], label='Train Accuracy Value')
plt.plot(h['val_accuracy'], label='Train Validation Accuracy Value')
plt.legend()
plt.show()